# 1.1 First NGSolve example

Let us solve the Poisson problem of finding $u$ satisfying 

$$
\begin{aligned}
-\Delta u & = f && \text { in  the unit square},
\\
u & = 0 && \text{ on the bottom and right parts of the boundary},
\\
\frac{\partial u }{\partial n } & = 0 
&& \text{ on the remaining  boundary parts}.
\end{aligned}
$$

## Quick steps to solution:

#### 1. Import NGSolve and Netgen Python modules:

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw

#### 2. Generate an unstructured mesh

In [2]:
mesh = Mesh(unit_square.GenerateMesh(maxh=1))
mesh.nv, mesh.ne   # number of vertices & elements 

(4, 2)

Here we prescribed a maximal mesh-size of 0.2 using the `maxh` flag. 

In [3]:
Draw(mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

#### 3. Declare a finite element space:

In [4]:
fes = H1(mesh, order=1, dirichlet="bottom|right")
fes.ndof  # number of unknowns in this space

4

Python's help system displays further documentation.

In [17]:
print(fes.FreeDofs())

0: 0001


In [18]:
help(fes)

Help on H1 in module ngsolve.comp object:

class H1(FESpace)
 |  An H1-conforming finite element space.
 |  
 |  The H1 finite element space consists of continuous and
 |  element-wise polynomial functions. It uses a hierarchical (=modal)
 |  basis built from integrated Legendre polynomials on tensor-product elements,
 |  and Jaboci polynomials on simplicial elements. 
 |  
 |  Boundary values are well defined. The function can be used directly on the
 |  boundary, using the trace operator is optional.
 |  
 |  The H1 space supports variable order, which can be set individually for edges, 
 |  faces and cells. 
 |  
 |  Internal degrees of freedom are declared as local dofs and are eliminated 
 |  if static condensation is on.
 |  
 |  The wirebasket consists of all vertex dofs. Optionally, one can include the 
 |  first (the quadratic bubble) edge basis function, or all edge basis functions
 |  into the wirebasket.
 |  
 |  Keyword arguments can be:
 |  
 |  order: int = 1
 |    order

#### 4. Declare test function, trial function, and grid function 

* Test and trial function are symbolic objects - called `ProxyFunctions` -  that help you construct bilinear forms (and have no space to hold solutions). 

* `GridFunctions`, on the other hand, represent functions in the finite element space and contains memory to hold coefficient vectors.

In [19]:
u = fes.TrialFunction()  # symbolic object
v = fes.TestFunction()   # symbolic object
gfu = GridFunction(fes)  # solution 

Alternately, you can get both the trial and test variables at once:

In [20]:
u, v = fes.TnT()

#### 5. Define and assemble linear and bilinear forms:

In [25]:
a = BilinearForm(fes)
a += grad(u)*grad(v)*dx
a.Assemble()

f = LinearForm(fes)
f += x*v*dx
f.Assemble();

# Assemble can be done using Gauss quadrature

Alternately, we can do one-liners: 

In [22]:
a = BilinearForm(grad(u)*grad(v)*dx).Assemble()
f = LinearForm(x*v*dx).Assemble()

You can examine the linear system in more detail:

In [23]:
print(f.vec)

   0.125
   0.125
 0.208333
 0.0416667




In [15]:
print(a.mat)

Row 0:   0: 1   1: -0.5   2: -1.11022e-16   3: -0.5
Row 1:   0: -0.5   1: 1   2: -0.5
Row 2:   0: -1.11022e-16   1: -0.5   2: 1   3: -0.5
Row 3:   0: -0.5   2: -0.5   3: 1



In [14]:
output_path = 'poisson_mat.txt'
with open(output_path, 'w') as file2:
    for j in range(a.mat.width):
        for i in range(a.mat.height):
#            print(a.mat[i,j])
            print(a.mat[i,j], file=file2)

#### 6. Solve the system:

In [24]:
gfu.vec.data = \
    a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec
Draw(gfu);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

The Dirichlet boundary condition constrains some degrees of freedom. The argument `fes.FreeDofs()` indicates that only the remaining "free" degrees of freedom should participate in the linear solve.

You can examine the coefficient vector of solution if needed:

In [26]:
print(gfu.vec)

       0
       0
       0
 0.0416667




You can see the zeros coming from the zero boundary conditions.

## Ways to interact with NGSolve

* A jupyter notebook (like this one) gives you one way to interact with NGSolve. When you have a complex sequence of tasks to perform, the notebook may not be adequate.


* You can write an entire python module in a text editor and call python on the command line. (A script of the above is provided in `poisson.py`.)
    ```
    python3 poisson.py
    ```
  
* If you want the Netgen GUI, then use `netgen` on the command line:
    ```
    netgen poisson.py
    ```
  You can then ask for a python shell from the GUI's menu options (`Solve -> Python shell`).
  